# Ejercicio - Busqueda de Alojamiento en Airbnb.

Supongamos que somos un agente de [Airbnb](http://www.airbnb.com) localizado en Lisboa, y tenemos que atender peticiones de varios clientes. Tenemos un archivo llamado `airbnb.csv` (en la carpeta data) donde tenemos información de todos los alojamientos de Airbnb en Lisboa.

In [2]:
import pandas as pd
df_airbnb = pd.read_csv("./data/airbnb.csv")

C:\Users\eduar\AppData\Local\Temp\ipykernel_21788\1394131765.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df_airbnb.head()

room_id  host_id        room_type       neighborhood  reviews  \
0     6499    14455  Entire home/apt              Belém        8   
1    17031    66015  Entire home/apt           Alvalade        0   
2    25659   107347  Entire home/apt  Santa Maria Maior       63   
3    29248   125768  Entire home/apt  Santa Maria Maior      225   
4    29396   126415  Entire home/apt  Santa Maria Maior      132   

   overall_satisfaction  accommodates  bedrooms  price  
0                   5.0             2       1.0   57.0  
1                   0.0             2       1.0   46.0  
2                   5.0             3       1.0   69.0  
3                   4.5             4       1.0   58.0  
4                   5.0             4       1.0   67.0

In [3]:
df_airbnb.dtypes

room_id                   int64
host_id                   int64
room_type                object
neighborhood             object
reviews                   int64
overall_satisfaction    float64
accommodates              int64
bedrooms                float64
price                   float64
dtype: object

In [4]:
df_airbnb.columns

Index(['room_id', 'host_id', 'room_type', 'neighborhood', 'reviews',
       'overall_satisfaction', 'accommodates', 'bedrooms', 'price'],
      dtype='object')

En concreto el dataset tiene las siguientes variables:
- room_id: el identificador de la propiedad
- host_id: el identificador del dueño de la propiedad
- room_type: tipo de propiedad (vivienda completa/(habitacion para compartir/habitación privada)
- neighborhood: el barrio de Lisboa
- reviews: El numero de opiniones
- overall_satisfaction: Puntuacion media del apartamento
- accommodates: El numero de personas que se pueden alojar en la propiedad
- bedrooms: El número de habitaciones
- price: El precio (en euros) por noche

## Usando Pandas

### Caso 1.

Alicia va a ir a Lisboa durante una semana con su marido y sus 2 hijos. Están buscando un apartamento con habitaciones separadas para los padres y los hijos. No les importa donde alojarse o el precio, simplemente quieren tener una experiencia agradable. Esto significa que solo aceptan lugares con más de 10 críticas con una puntuación mayor de 4. Cuando seleccionemos habitaciones para Alicia, tenemos que asegurarnos de ordenar las habitaciones de mejor a peor puntuación. Para aquellas habitaciones que tienen la misma puntuación, debemos mostrar antes aquellas con más críticas. Debemos darle 3 alternativas.

In [6]:
# Filtrar apartamento que cumplan con criterios: (criticas > 10 & puntuacion > 4)
a_filtrados = df_airbnb[ (df_airbnb['reviews'] > 10) & (df_airbnb['overall_satisfaction'] > 4) ]

# Ordenar los apartamento filtrados por puntuación media (mejor a poer) y luego según número de críticas(mayor cantidad)
a_ordenados = a_filtrados.sort_values(by=['overall_satisfaction', 'reviews'], ascending=[False, False]) #decreser

# Paso 3: Mostrar las tres mejores opciones
mejores_3opciones = a_ordenados.head(3)

# Mostrar las tres mejores opciones a Alicia
print("Las tres mejores opciones para Alicia son:")
print(mejores_3opciones[['room_id', 'room_type', 'neighborhood', 'reviews', 'overall_satisfaction', 'price']])

Las tres mejores opciones para Alicia son:
     room_id        room_type   neighborhood  reviews  overall_satisfaction  \
120   176153  Entire home/apt   Misericórdia      438                   5.0   
931  1085634  Entire home/apt   Misericórdia      379                   5.0   
212   299321  Entire home/apt  Santo António      337                   5.0   

     price  
120  102.0  
931   52.0  
212   87.0  


### Caso 2

Roberto es un casero que tiene una casa en Airbnb. De vez en cuando nos llama preguntando sobre cuales son las críticas de su alojamiento. Hoy está particularmente enfadado, ya que su hermana Clara ha puesto una casa en Airbnb y Roberto quiere asegurarse de que su casa tiene más críticas que las de Clara. Tenemos que crear un dataframe con las propiedades de ambos. Las id de las casas de Roberto y Clara son 97503 y 90387  respectivamente. Finalmente guardamos este dataframe como excel llamado "roberto.xls

In [5]:
# Paso 1: Filtrar propiedades de Roberto y Clara
condicion_roberto = df_airbnb['room_id'] == 97503
propiedades_roberto = df_airbnb[condicion_roberto]
propiedades_roberto.head()

condicion_clara = df_airbnb['room_id'] == 90387
propiedades_clara = df_airbnb[condicion_clara]
propiedades_clara.head()

# Paso 2: Crear un nuevo DataFrame con las propiedades de ambos
propiedades_roberto_clara = pd.concat([propiedades_roberto, propiedades_clara], axis=0)

# Paso 3: Guardar el nuevo DataFrame como un archivo Excel
propiedades_roberto_clara.to_excel("roberto.xlsx", index=False)

print("Se ha creado el archivo 'roberto.xls' con las propiedades de Roberto y Clara.")

Se ha creado el archivo 'roberto.xls' con las propiedades de Roberto y Clara.



### Caso 3

Diana va a Lisboa a pasar 3 noches y quiere conocer a gente nueva. Tiene un presupuesto de 50€ para su alojamiento. Debemos buscarle las 10 propiedades más baratas, dandole preferencia a aquellas que sean habitaciones compartidas *(room_type == Shared room)*, y para aquellas viviendas compartidas debemos elegir aquellas con mejor puntuación.

In [33]:
# Paso 1: Filtrar propiedades que cumplan con los criterios: (presupuesto <= 50) & (conocer nueva gente)
condicion_presupuesto = df_airbnb['price'] <= 50
condicion_npersonas = df_airbnb['accommodates'] >= 2
filtro1 = df_airbnb[ (condicion_presupuesto) & (condicion_npersonas) ]

# Paso 2: Dar preferencia a habitaciones compartidas
condicion_tipo = filtro1['room_type'] == 'Shared room'
filtro2 = filtro1[ condicion_tipo ]

# Paso 3: Ordenar habitaciones compartidas por puntuación de mejor a peor, si las hay
if not filtro2.empty:
    filtro2_ordenadas = filtro2.sort_values(by='overall_satisfaction', ascending=False)
    propiedades_seleccionadas = filtro2_ordenadas.head(10)
else:
    # Si no hay habitaciones compartidas, simplemente ordenar por precio y seleccionar las primeras 10
    filtro2_ordenadas = filtro2.sort_values(by='price', ascending=False)
    propiedades_seleccionadas = filtro2_ordenadas.head(10)

# Mostrar las propiedades seleccionadas a Diana
print("Las 10 propiedades escogidas para Diana son:")
print(propiedades_seleccionadas[['room_id', 'room_type', 'neighborhood', 'overall_satisfaction', 'price']])

Las 10 propiedades escogidas para Diana son:
       room_id    room_type    neighborhood  overall_satisfaction  price
6399  11278059  Shared room         Arroios                   5.0   22.0
1656   2382263  Shared room     São Vicente                   5.0   31.0
7932  13570032  Shared room    Misericórdia                   5.0   48.0
4911   7722107  Shared room  Avenidas Novas                   5.0   36.0
8918  14719962  Shared room  Avenidas Novas                   5.0   13.0
7552  13078189  Shared room           Ajuda                   5.0   21.0
3727   5803348  Shared room    Misericórdia                   5.0   20.0
9065  14933182  Shared room   Santo António                   5.0   11.0
9964  16150682  Shared room  Avenidas Novas                   5.0   16.0
6527  11523225  Shared room  Avenidas Novas                   5.0   23.0
